In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
from privacypacking.utils.utils import load_logs, global_metrics
import pandas as pd
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from experiments.ray.analysis import get_percentiles

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_22-58-03"))

In [ ]:
rdf.columns

In [ ]:
rdf["mean_task_per_block"].unique()

In [ ]:
(rdf["maximum_profit"] / rdf["n_tasks"]).unique()

In [ ]:
px.line(
    rdf.query("n_tasks == maximum_profit and scheduler_metric == 'SoftKnapsack' and mean_task_per_block == 250 ").sort_values("temperature"),
    x="temperature",
    y="n_allocated_tasks",
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,1200],
    log_x=True,
    title="10 + 20 blocks, privatekube. No profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
px.line(
    rdf.query("n_tasks == maximum_profit and metric_recomputation_period == 10 and temperature == 0.01").sort_values(["mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,1200],
    title="10 + 20 blocks, privatekube. No profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf["profits"] = rdf.config.apply(lambda d: d["tasks_spec"]["curve_distributions"]["custom"]["data_path"][-5:])

In [ ]:
rdf.profits.unique()

In [ ]:
px.line(
    rdf.query("profits == '=size' and metric_recomputation_period == 10 and temperature == 0.01").sort_values(["mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    facet_col="profits",
    facet_col_wrap=1,
    # range_y=[0,1200],
    title="10 + 20 blocks, privatekube, offline.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
px.line(
    rdf.query("profits == '5_p=1' and metric_recomputation_period == 10 and temperature == 0.01").sort_values(["mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    facet_col="profits",
    facet_col_wrap=1,
    # range_y=[0,1200],
    title="10 + 20 blocks, privatekube, offline.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
px.line(
    rdf.query("profits == 'ksize' and metric_recomputation_period == 10 and temperature == 0.01").sort_values(["mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    facet_col="profits",
    facet_col_wrap=1,
    # range_y=[0,1200],
    title="10 + 20 blocks, privatekube, offline.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_09-41-19"))

In [ ]:
fig = px.line(
    rdf.sort_values(["temperature", "data_path"]),
    x="temperature",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    log_x=True,
    facet_col="data_path",
    facet_col_wrap=1,
    # range_y=[0,1200],
    title="10 + 20 blocks, privatekube, offline.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)
fig.layout.yaxis1.update(matches=None)
fig.layout.yaxis2.update(matches=None)
fig.layout.yaxis3.update(matches=None)
fig

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_11-58-57"))
# What is the impact of capping the demands? + double check load

In [ ]:
rdf.config.iloc[0]

In [ ]:
rdf["clip"] =  rdf.config.apply(lambda d: d["omegaconf"]["metric"]["clip_demands_in_relevance"])

In [ ]:
rdf.total_tasks.unique()

In [ ]:
fig = px.line(
    rdf.sort_values(["mean_task_per_block", "data_path"]),
    x="mean_task_per_block",
    y="n_allocated_tasks",
    color="clip",
    # log_x=True,
    width=800,
    height=600,
    log_x=True,
    facet_col="data_path",
    facet_col_wrap=1,
    # range_y=[0,1200],
    title="10 + 20 blocks, privatekube, offline. <br> Temp 0.1, norm by capacity. No profits. ",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)
fig

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_11-15-18"))


In [ ]:
rdf

In [ ]:
fig = px.line(
    rdf.sort_values(["temperature", "data_path"]),
    x="temperature",
    y="n_allocated_tasks",
    # color="clip",
    # log_x=True,
    width=800,
    height=600,
    log_x=True,
    facet_col="data_path",
    facet_col_wrap=1,
    # range_y=[0,1200],
    title="10 + 20 blocks, privatekube, offline. <br> No profits. 600 mean tasks per block",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)
fig

Nice! The optimal temperature doesn't seem to depend on the load too much. But it does depend on the profits.

- [ ] Check if it depends on the number of blocks

In [ ]:
# Now, try different loads and see if we observe the slight dominance of SK over other schedulers. 
# No profits. Temperature = 0.1 (the best for this profit profile, at least for mean task 300 and 600)
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_15-03-37"))

px.line(
    rdf.sort_values(["mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, offline.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Same but with profits = size.
# Temp = 0.01 (the best for this profit profile, at least for mean task 300 and 600)
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_15-09-25"))

px.line(
    rdf.sort_values(["mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, offline. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Temperature tuning for SK with two different load, with grid profits. Amplitude: 1000. Shift: 10.
# - [ ] Smaller amplitude but bigger shift?

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_15-27-59"))

px.line(
    rdf.sort_values(["temperature", "mean_task_per_block"]),
    x="temperature",
    y="realized_profit",
    color="mean_task_per_block",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    log_x=True,
    title="10 + 20 blocks, privatekube, offline. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Temperature tuning for SK with two different load, with grid profits. 
# Integer grid. Amplitude: 500. Shift: 10.                     profit = "500:0.25, 100:0.25, 50:0.25, 10:0.25" / profit = "50:0.25, 10:0.25, 5:0.25, 1:0.25"

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_16-24-54"))

px.line(
    rdf.sort_values(["temperature", "mean_task_per_block"]),
    x="temperature",
    y="realized_profit",
    color="mean_task_per_block",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    log_x=True,
    title="10 + 20 blocks, privatekube, offline. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Check how it looks like when I increase the number of tasks. And compare with other metrics
# Integer grid profits. Temperature: 1e-4 (the best)
# - [ ] If it looks good, repeat and average a few seeds (if not smooth). Then, play with T and lifetime.
# - [x] Sprinkle some Gaussian curves?? They seem pretty much useless (free). Unless I purposely give them profit 50x smaller than laplace, which is suspicious.
# - [WIP] Play with the mice ratio?

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_16-37-51"))

px.line(
    rdf.sort_values(["mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, offline. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Same seed as above, just normalizing by available budget instead of capacity.
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_17-17-08"))

px.line(
    rdf.sort_values(["mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, offline. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Same as previous but with a different seed (1)
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_18-37-31"))

px.line(
    rdf.sort_values(["mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, offline. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Playing with the ratio! Seed 0, normalize by capacity. Warning: the optimal temperature might have changed? Maybe I should just compute it on paper.
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_17-31-39"))

px.line(
    rdf.sort_values(["data_path", "mean_task_per_block", "scheduler_metric"]),
    x="mean_task_per_block",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    facet_col="data_path",
    facet_col_wrap=1,    
    title="10 + 20 blocks, privatekube, offline. Int grid Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)



In [ ]:
# Online, grid profits, 0.5 mice. L = 10. Different T.

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_18-49-20"))

px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, online. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Online, grid profits, 0.5 mice. L = 10. Different T. Different load: 500 instead of 700. 
# WARNING: normalize by capacity and not remaining budget!

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_20-12-28"))

px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, online. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf.scheduling_times

In [ ]:
# MASSIVE! Same as above but with normalization by available. And for both 500 and 1000 tasks per block, with more range for T.

# TODO: 
# - [x] high refresh rate for 50. Try 5 and 1. (see below)
# - [x] More low T, no need for more than 40. (see high cpu machine)
# - [ ] Plot some runtimes
# - [ ] The single-block reduction!

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_21-38-25"))

px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=1600,
    height=600,
    range_y=[0,1.2e6],
    log_x=True,
    facet_col="mean_task_per_block",
    facet_col_wrap=2,
    title="10 + 20 blocks, privatekube, online. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# rdf.iloc[0].config

In [ ]:
# Different step sizes
# - [ ] Try step 10 and 25

rdf50 = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_21-38-25")).query("T in [1, 20]")
rdf1_5 = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_07-24-37"))
rdf = pd.concat([rdf50, rdf1_5])
px.line(
    rdf.query("mean_task_per_block == 500").sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=1600,
    height=600,
    hover_data=["mean_task_per_block"],
    # range_y=[0,15_000],
    facet_col="metric_recomputation_period",
    facet_col_wrap=2,
    title="10 + 20 blocks, privatekube, online. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Another CRAZY one. Realistic time frames. . Just plot T.
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_21-50-57"))

px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=1600,
    height=600,
    # range_y=[0,15_000],
    facet_col="mean_task_per_block",
    facet_col_wrap=2,
    title="10 + 20 blocks, privatekube, online. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# Online, grid profits, 0.5 mice. L = 10. Different T. Different load: 500 instead of 700. Warning: normalize by capacity and not remaining budget!

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_20-12-28"))

px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, online. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
print(rdf.normalize_by.unique(), rdf.n_tasks.unique(), rdf.mean_task_per_block.unique())

In [ ]:
# Online, grid profits, 0.5 mice. L = 10. Different T.
# WARNING: normalize by capacity and not remaining budget!
# Exactly same one, but with a 5-steps refresh (to see if it's better than 50) and more T.
# CONCLUSION: significant improvement. Refreshing every 50 steps is too coarse (so the other figures can get even better, if we give them more time).
# All the graphs are shifted (excpect FCFS), but DPF benefits less from the shift.
# - [x] Any other lifetime that would make sense?
# - [x] More blocks, e.g. 50 online blocks?
# - [x] Just 500 tasks?
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-08_20-09-52"))

px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, online. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# On the fast CPU (rsync'ed)
# Some very small T
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_14-30-40"))
print(rdf.metric_recomputation_period.unique(), rdf.normalize_by.unique(), rdf.data_path.unique(), rdf.mean_task_per_block.unique())
px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    log_x=True,
    width=800,
    height=600,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, online. Profits.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_14-30-40"))
rdf.columns

In [ ]:
# On the fast CPU (rsync'ed)
# Intermediate metric steps, with and without profits.
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_18-13-06"))
# .query("'grid' in data_path")
print(rdf.metric_recomputation_period.unique(), rdf.normalize_by.unique(), rdf.data_path.unique(), rdf.mean_task_per_block.unique())
# print("=1" in rdf.data_path.iloc[0])
fig = px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    log_x=True,
    width=1200,
    height=1000,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, online. With and without profits.",
    facet_col="metric_recomputation_period",
    facet_row="data_path",
    facet_col_wrap=2,
)

fig.layout.yaxis2.update(matches=None)
fig.layout.yaxis1.update(matches=None)

fig.layout.yaxis3.update(matches=None)
fig.layout.yaxis4.update(matches=None)
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"

fig

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-09_18-13-06"))
rdf.columns
rdf["avg_delay"] = rdf.allocated_tasks_scheduling_delays.apply(lambda x: np.mean(x))
rdf["p95"] = rdf.allocated_tasks_scheduling_delays.apply(lambda x: np.percentile(x, 95))

fig = px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="avg_delay",
    color="scheduler_metric",
    # log_x=True,
    width=1200,
    height=1000,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, online. With and without profits.",
    facet_col="metric_recomputation_period",
    facet_row="data_path",
    facet_col_wrap=2,
)

# fig.layout.yaxis2.update(matches=None)
# fig.layout.yaxis1.update(matches=None)

# fig.layout.yaxis3.update(matches=None)
# fig.layout.yaxis4.update(matches=None)
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"

fig

In [ ]:
fig = px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="p95",
    color="scheduler_metric",
    # log_x=True,
    width=1200,
    height=1000,
    # range_y=[0,15_000],
    title="10 + 20 blocks, privatekube, online. With and without profits.",
    facet_col="metric_recomputation_period",
    facet_row="data_path",
    facet_col_wrap=2,
)

# fig.layout.yaxis2.update(matches=None)
# fig.layout.yaxis1.update(matches=None)

# fig.layout.yaxis3.update(matches=None)
# fig.layout.yaxis4.update(matches=None)
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"

fig

In [ ]:
# Obsolete below

In [ ]:
# Tuning
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_10-32-11"))
rdf.columns
px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)


In [ ]:
# Tuning
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_14-28-33"))
rdf.columns
px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf.data_path.unique()

In [ ]:
# Temp tuning with profits
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_14-55-57"))
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-04-36"))

px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-08-13"))

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-18-48"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-52-12"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline.  Poly ratio. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_18-34-39"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline. Softmax with subtracting the max.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_19-32-46"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Softmax with subtracting the max.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_20-34-49"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Softmax with subtracting the max.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [ ]:
# On privatekube now!!


rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-29-50"))
rdf = pd.concat([rdf, load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-42-40"))])
px.bar(
    rdf.sort_values(["T", "scheduler_metric"]),
    # x="T",
    x="scheduler_metric",    
    y="realized_profit",
    # color="scheduler_metric",
    # log_x=True,
    width=1000,
    height=600,
    range_y=[0,6_000],
    title="10 initial blocks + 20 online blocks, with profits, PrivateKube. Unlocking with lifetime 10. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)


In [ ]:
rdf = load_ray_experiment(Path("run_and_report_2022-03-27_19-45-38"))
# rdf = pd.concat([rdf, load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-43-03"))])
rdf
# px.bar(
#     rdf.sort_values(["T", "scheduler_metric"]),
#     x="scheduler_metric",
#     y="n_allocated_tasks",
#     # log_x=True,
#     width=1000,
#     height=600,
#     range_y=[0,10_000],
#     title="10 initial blocks + 20 online blocks, NO profits, PrivateKube. Unlocking with lifetime 10. Temperature = 1",
#     # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
# )